In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime
import random

In [0]:
dbutils.widgets.text("catalog", "workspace_bank", "Catalog name")
dbutils.widgets.text("silver_schema", "silver", "Silver schema name")
dbutils.widgets.text("gold_schema", "gold", "Gold schema name")
dbutils.widgets.text("silver_transactions_table", "consolidated_credit_card_merchant_trx", "Silver transactions table")
dbutils.widgets.text("silver_campaigns_table", "marketing_campaigns", "Silver campaigns table")
dbutils.widgets.text("gold_table", "credit_card_sales_summary_mc", "Gold table name")
dbutils.widgets.text("processing_date", "", "Processing Date (YYYY-MM-DD) - Leave empty for current date")
dbutils.widgets.text("seasonal_variance_min", "0.01", "Min seasonal variance (0.01 = 1%)")
dbutils.widgets.text("seasonal_variance_max", "0.40", "Max seasonal variance (0.40 = 40%)")

# Get parameters
catalog = dbutils.widgets.get("catalog")
silver_schema = dbutils.widgets.get("silver_schema")
gold_schema = dbutils.widgets.get("gold_schema")
silver_transactions_table = dbutils.widgets.get("silver_transactions_table")
silver_campaigns_table = dbutils.widgets.get("silver_campaigns_table")
gold_table = dbutils.widgets.get("gold_table")
processing_date_param = dbutils.widgets.get("processing_date")
seasonal_variance_min = float(dbutils.widgets.get("seasonal_variance_min"))
seasonal_variance_max = float(dbutils.widgets.get("seasonal_variance_max"))

# Generate ETL metadata
etl_datetime = datetime.now()
etl_date = etl_datetime.strftime("%Y-%m-%d")
etl_time = etl_datetime.strftime("%H:%M:%S")

# Set processing date (business date)
if processing_date_param == "":
    processing_date = etl_date
else:
    processing_date = processing_date_param

# Full table names
silver_transactions_full = f"{catalog}.{silver_schema}.{silver_transactions_table}"
silver_campaigns_full = f"{catalog}.{silver_schema}.{silver_campaigns_table}"
gold_table_full = f"{catalog}.{gold_schema}.{gold_table}"

print("="*80)
print(f"📊 GOLD LAYER ETL - CREDIT CARD SALES SUMMARY")
print("="*80)
print(f"Source (Transactions): {silver_transactions_full}")
print(f"Source (Campaigns): {silver_campaigns_full}")
print(f"Target: {gold_table_full}")
print(f"Processing Date (Business Date): {processing_date}")
print(f"ETL Date (Job Run Date): {etl_date}")
print(f"ETL Time: {etl_time}")
print(f"Seasonal Variance Range: {seasonal_variance_min} to {seasonal_variance_max}")
print("="*80)

In [0]:
def calculate_seasonal_baseline(intra_day_sales, seed, var_min, var_max):
    """
    Calculate avg_seasonal_daily_total_sales based on intra_day_total_sales
    
    Formula: intra_day_sales * (1 +/- random_pct)
    where random_pct is between var_min (0.01 = 1%) and var_max (0.40 = 40%)
    
    Parameters:
    - intra_day_sales: Current day's total sales from aggregation
    - seed: Random seed for reproducibility
    - var_min: Minimum variance percentage (e.g., 0.01 = 1%)
    - var_max: Maximum variance percentage (e.g., 0.40 = 40%)
    
    Returns:
    - Seasonal baseline = intra_day_sales * (1 +/- random_pct)
    """
    if intra_day_sales is None or intra_day_sales == 0:
        return 0.0
    
    random.seed(seed)
    
    # Random percentage between var_min and var_max (e.g., 0.01 to 0.40)
    random_pct = random.uniform(var_min, var_max)
    
    # Random direction: +/- (50/50 chance)
    direction = 1 if random.random() < 0.5 else -1
    
    # Calculate: intra_day_sales * (1 +/- random_pct)
    seasonal_baseline = intra_day_sales * (1 + (direction * random_pct))
    
    return round(seasonal_baseline, 2)

# Register UDF
seasonal_baseline_udf = F.udf(calculate_seasonal_baseline, DoubleType())

print("✅ UDF registered: calculate_seasonal_baseline")

In [0]:
print(f"\n📖 Reading today's transactions from Silver layer...")
print(f"   Filter: _silver_processing_date = '{processing_date}'")

# Read all of today's transactions from Silver
df_today_transactions = (
    spark.read
    .table(silver_transactions_full)
    .filter(F.col("_silver_processing_date") == processing_date)
)

# Count records
transaction_count = df_today_transactions.count()
print(f"✅ Loaded {transaction_count:,} transactions for {processing_date}")

In [0]:
print(f"\n📖 Reading current active campaigns...")
print(f"   Filter: campaign_status = 'ACTIVE' AND partition_date = '{processing_date}'")

# Get the latest partition date from campaigns table
latest_campaign_partition = (
    spark.read.table(silver_campaigns_full)
    .select(F.max("partition_date").alias("max_date"))
    .collect()[0]["max_date"]
)

print(f"   Latest campaign partition: {latest_campaign_partition}")

# Read current active campaigns for CATEGORY target type
df_active_campaigns = (
    spark.read.table(silver_campaigns_full)
    .filter(F.col("partition_date") == latest_campaign_partition)
    .filter(F.col("campaign_status") == "ACTIVE")
    .filter(F.col("target_type") == "CATEGORY")
    .select(
        "merchant_category",
        "campaign_id",
        "campaign_type",
        "campaign_status",
        "target_uplift"
    )
)


In [0]:
print(f"\n🔗 Joining transactions with active campaigns...")

# Left join - not all merchant categories have active campaigns
df_with_campaigns = (
    df_today_transactions
    .join(
        df_active_campaigns,
        "merchant_category",
        "left"
    )
)

joined_count = df_with_campaigns.count()
print(f"✅ Joined dataset: {joined_count:,} records")

In [0]:
print(f"\n📊 Aggregating by merchant_category...")

# Aggregate entire day's data
df_aggregated = (
    df_with_campaigns
    .groupBy("merchant_category")
    .agg(
        # Intra-day total sales: sum of transaction amounts
        F.sum("transaction_amount").alias("intra_day_total_sales"),
        
        # Campaign exist flag: TRUE if any ACTIVE campaign exists for this category
        F.max(
            F.when(F.col("campaign_status") == "ACTIVE", True).otherwise(False)
        ).alias("campaign_exist_flag"),
        
        # Target uplift: get the uplift target (should be consistent within category)
        F.first("target_uplift", ignorenulls=True).alias("target_uplift")
    )
)

# Count categories
category_count = df_aggregated.count()
print(f"✅ Aggregated into {category_count:,} merchant categories")

In [0]:
print(f"\n🎲 Calculating seasonal baseline with random variance...")

# Generate seed for seasonal baseline calculation (based on merchant_category hash + date)
df_with_baseline = (
    df_aggregated
    .withColumn(
        "seed",
        F.abs(F.hash(F.col("merchant_category"), F.lit(processing_date)))
    )
    .withColumn(
        "avg_seasonal_daily_total_sales",
        seasonal_baseline_udf(
            F.col("intra_day_total_sales"),
            F.col("seed"),
            F.lit(seasonal_variance_min),
            F.lit(seasonal_variance_max)
        )
    )
    .drop("seed")
)

print(f"✅ Seasonal baselines calculated")

In [0]:
print(f"\n📝 Adding metadata columns...")

# Add partition_date and ETL metadata
df_final = (
    df_with_baseline
    .withColumn("partition_date", F.lit(processing_date).cast(DateType()))
    .withColumn("etl_date", F.lit(etl_date).cast(DateType()))
    .withColumn("etl_time", F.lit(etl_time).cast(StringType()))
)

# Reorder columns
df_final = df_final.select(
    "merchant_category",
    "intra_day_total_sales",
    "avg_seasonal_daily_total_sales",
    "campaign_exist_flag",
    "target_uplift",
    "partition_date",
    "etl_date",
    "etl_time"
)

print(f"✅ Final schema prepared")

In [0]:
print(f"\n💾 Writing to Gold table: {gold_table_full}")
print(f"   Mode: Overwrite partition {processing_date}")

# Write to gold table - overwrite only today's partition
(
    df_final
    .write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"partition_date = '{processing_date}'")
    .partitionBy("partition_date")
    .saveAsTable(gold_table_full)
)

print(f"✅ Successfully wrote to partition {processing_date}")

In [0]:
# Query 1: Categories with active campaigns and high uplift targets
print("\n🎯 Query 1: Active Campaigns with Target Uplift > 50%")
print("-" * 80)

df_high_uplift = spark.sql(f"""
    SELECT 
        merchant_category,
        intra_day_total_sales,
        avg_seasonal_daily_total_sales,
        ROUND((intra_day_total_sales / avg_seasonal_daily_total_sales - 1) * 100, 2) as actual_uplift_pct,
        target_uplift,
        ROUND(target_uplift * 100, 2) as target_uplift_pct,
        campaign_exist_flag,
        partition_date
    FROM {gold_table_full}
    WHERE partition_date = '{processing_date}'
        AND campaign_exist_flag = TRUE
        AND target_uplift > 0.50
    ORDER BY target_uplift DESC, intra_day_total_sales DESC
""")

df_high_uplift.show(20, truncate=False)

# Query 2: Top performing categories by sales
print("\n💰 Query 2: Top 10 Categories by Intra-Day Sales")
print("-" * 80)

df_top_sales = spark.sql(f"""
    SELECT 
        merchant_category,
        intra_day_total_sales,
        avg_seasonal_daily_total_sales,
        campaign_exist_flag,
        target_uplift,
        partition_date
    FROM {gold_table_full}
    WHERE partition_date = '{processing_date}'
    ORDER BY intra_day_total_sales DESC
    LIMIT 10
""")

df_top_sales.show(truncate=False)

# Query 3: Categories performing above seasonal baseline
print("\n📈 Query 3: Categories Performing Above Seasonal Baseline")
print("-" * 80)

df_above_baseline = spark.sql(f"""
    SELECT 
        merchant_category,
        intra_day_total_sales,
        avg_seasonal_daily_total_sales,
        ROUND(intra_day_total_sales - avg_seasonal_daily_total_sales, 2) as difference,
        ROUND((intra_day_total_sales / avg_seasonal_daily_total_sales - 1) * 100, 2) as pct_above_baseline,
        campaign_exist_flag,
        partition_date
    FROM {gold_table_full}
    WHERE partition_date = '{processing_date}'
        AND intra_day_total_sales > avg_seasonal_daily_total_sales
    ORDER BY pct_above_baseline DESC
    LIMIT 10
""")

df_above_baseline.show(truncate=False)

print("="*80)


# Query 4: Categories performing below seasonal baseline
print("\n📈 Query 4: Categories Performing Below Seasonal Baseline")
print("-" * 80)

df_above_baseline = spark.sql(f"""
    SELECT 
        merchant_category,
        intra_day_total_sales,
        avg_seasonal_daily_total_sales,
        ROUND(intra_day_total_sales - avg_seasonal_daily_total_sales, 2) as difference,
        ROUND((intra_day_total_sales / avg_seasonal_daily_total_sales - 1) * 100, 2) as pct_above_baseline,
        campaign_exist_flag,
        partition_date
    FROM {gold_table_full}
    WHERE partition_date = '{processing_date}'
        AND intra_day_total_sales < avg_seasonal_daily_total_sales
    ORDER BY pct_above_baseline DESC
    LIMIT 10
""")

df_above_baseline.show(truncate=False)

print("="*80)

# ETL Completion Summary

print("\n" + "="*80)
print("🎉 ETL COMPLETED SUCCESSFULLY")
print("="*80)
print(f"Processing Date: {processing_date}")
print(f"ETL Date: {etl_date}")
print(f"ETL Time: {etl_time}")
print(f"Source Transactions: {transaction_count:,} records")
print(f"Target Table: {gold_table_full}")
print("="*80)
print("\n✅ Gold layer is ready for dashboard consumption!")
print("\n📊 Dashboard Query:")
print(f"   SELECT * FROM {gold_table_full} WHERE partition_date = CURRENT_DATE()")